# 1.4 特雷诺比率 (Treynor Ratio)

## 概念介绍

特雷诺比率(Treynor Ratio)由杰克·特雷诺(Jack Treynor)于1965年提出，是一种风险调整后的绩效评估指标。与夏普比率类似，特雷诺比率也衡量投资组合超额收益与风险的关系，但不同之处在于，特雷诺比率使用Beta(β)作为风险度量，而不是标准差。

### 特雷诺比率公式

$Treynor\ Ratio = \frac{R_p - R_f}{\beta_p}$

其中：
- $R_p$ 是投资组合的期望收益率
- $R_f$ 是无风险收益率
- $\beta_p$ 是投资组合相对于市场的Beta系数

Beta系数衡量了投资组合收益对市场变动的敏感性，是投资组合与市场的协方差除以市场方差：

$\beta_p = \frac{Cov(R_p, R_m)}{Var(R_m)}$

特雷诺比率越高，表示单位系统性风险下的超额收益越高，意味着投资组合在面对市场风险时的表现越好。

## 1.1 安装必要的库

首先安装需要用到的Python库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib statsmodels

## 1.2 从Github加载数据

In [ ]:

!wget -O AAPL_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/AAPL_data.csv"
!wget -O SPY_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/SPY_data.csv"
file_path_aapl = "AAPL_data.csv"
file_path_spy = "SPY_data.csv"

## 1.3 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 1.4 数据加载和预处理

In [ ]:
# 设定无风险收益率
risk_free_rate = 0.02 / 252  # 假设年利率为 2%，转换为日利率

# 读取数据
try:
    # 读取 CSV 并跳过前两行
    column_names = ["Date", "Open", "High", "Low", "Close", "Volume"]
    
    data_aapl = pd.read_csv(file_path_aapl, skiprows=3, names=column_names)
    data_spy = pd.read_csv(file_path_spy, skiprows=3, names=column_names)
    print("数据加载成功")
except Exception as e:
    print(f"数据加载失败: {e}")
    # 如果加载失败，尝试不跳过行读取
    try:
        data_aapl = pd.read_csv(file_path_aapl)
        data_spy = pd.read_csv(file_path_spy)
        print("重新尝试数据加载成功")
    except Exception as e:
        print(f"数据加载再次失败: {e}")

In [ ]:
# 数据预览
print("AAPL数据预览:")
display(data_aapl.head())
print("\nSPY数据预览:")
display(data_spy.head())

In [ ]:
# 统一列名（将 "Close" 重命名为 "Adj Close"）
data_aapl.rename(columns={'Close': 'Adj Close'}, inplace=True)
data_spy.rename(columns={'Close': 'Adj Close'}, inplace=True)

# 转换日期格式并设置索引
try:
    data_aapl['Date'] = pd.to_datetime(data_aapl['Date'], format="%Y/%m/%d")
except Exception as e:
    print(f"AAPL日期转换错误，尝试自动检测格式: {e}")
    data_aapl['Date'] = pd.to_datetime(data_aapl['Date'])

try:    
    data_spy['Date'] = pd.to_datetime(data_spy['Date'], format="%Y/%m/%d")
except Exception as e:
    print(f"SPY日期转换错误，尝试自动检测格式: {e}")
    data_spy['Date'] = pd.to_datetime(data_spy['Date'])

data_aapl.set_index('Date', inplace=True)
data_spy.set_index('Date', inplace=True)

In [ ]:
# 计算每日收益率
data_aapl['Daily Return'] = data_aapl['Adj Close'].pct_change()
data_spy['Daily Return'] = data_spy['Adj Close'].pct_change()

# 清除缺失值
data_aapl = data_aapl.dropna()
data_spy = data_spy.dropna()

# 数据基本统计量
print("AAPL日收益率统计:")
display(data_aapl['Daily Return'].describe())
print("\nSPY日收益率统计:")
display(data_spy['Daily Return'].describe())

## 1.5 计算Beta系数

In [ ]:
# 确保两个DataFrame具有相同的索引
# 寻找两个数据集的共同日期
common_dates = data_aapl.index.intersection(data_spy.index)
data_aapl_aligned = data_aapl.loc[common_dates]
data_spy_aligned = data_spy.loc[common_dates]

# 验证对齐后的数据大小
print(f"AAPL对齐后数据量: {len(data_aapl_aligned)}")
print(f"SPY对齐后数据量: {len(data_spy_aligned)}")

In [ ]:
# 计算Beta值（AAPL与SPY的回归分析）
# 准备回归数据
X = sm.add_constant(data_spy_aligned['Daily Return'])  # 加入常数项（截距）
y = data_aapl_aligned['Daily Return']

# 运行回归模型
model = sm.OLS(y, X).fit()

# 提取Beta系数
beta_aapl = model.params[1]

# 显示回归结果摘要
print(model.summary())

## 1.6 计算特雷诺比率

In [ ]:
# 计算AAPL和SPY的平均回报（日回报）
mu_aapl = data_aapl_aligned['Daily Return'].mean()
mu_spy = data_spy_aligned['Daily Return'].mean()

# 计算年化收益率
annual_mu_aapl = mu_aapl * 252
annual_mu_spy = mu_spy * 252
annual_risk_free = risk_free_rate * 252

# 计算Treynor比率
treynor_aapl = (mu_aapl - risk_free_rate) / beta_aapl
treynor_spy = (mu_spy - risk_free_rate) / 1  # SPY的Beta值假定为1，因为SPY是市场本身

# 计算年化Treynor比率
annual_treynor_aapl = (annual_mu_aapl - annual_risk_free) / beta_aapl
annual_treynor_spy = (annual_mu_spy - annual_risk_free) / 1

# 打印计算结果
print("日度计算结果:")
print(f"AAPL的Beta值: {beta_aapl:.4f}")
print(f"AAPL的每日平均回报: {mu_aapl:.4f}")
print(f"AAPL的Treynor比率: {treynor_aapl:.4f}")

print(f"SPY的Beta值: 1.0000 (基准)")
print(f"SPY的每日平均回报: {mu_spy:.4f}")
print(f"SPY的Treynor比率: {treynor_spy:.4f}")

print("\n年化计算结果:")
print(f"AAPL的年化平均回报: {annual_mu_aapl:.4f} ({annual_mu_aapl:.2%})")
print(f"AAPL的年化Treynor比率: {annual_treynor_aapl:.4f}")
print(f"SPY的年化平均回报: {annual_mu_spy:.4f} ({annual_mu_spy:.2%})")
print(f"SPY的年化Treynor比率: {annual_treynor_spy:.4f}")

## 1.7 结果可视化

In [ ]:
# 回归拟合可视化
plt.figure(figsize=(12, 8))

# 散点图：市场收益率vs股票收益率
plt.scatter(data_spy_aligned['Daily Return'], data_aapl_aligned['Daily Return'], 
            alpha=0.5, label='日收益率数据点')

# 回归线
x_range = np.linspace(data_spy_aligned['Daily Return'].min(), data_spy_aligned['Daily Return'].max(), 100)
y_pred = model.params[0] + model.params[1] * x_range
plt.plot(x_range, y_pred, 'r', linewidth=2, label=f'回归线 (Beta = {beta_aapl:.4f})')

# 添加市场线（Beta=1）
y_market = model.params[0] + 1 * x_range
plt.plot(x_range, y_market, 'g--', linewidth=2, label='市场线 (Beta = 1)')

# 设置图表属性
plt.title('AAPL与市场(SPY)收益率回归分析', fontsize=16)
plt.xlabel('市场(SPY)日收益率', fontsize=12)
plt.ylabel('AAPL日收益率', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# 添加注释
plt.annotate(f"Beta = {beta_aapl:.4f}\nR² = {model.rsquared:.4f}", 
             xy=(0.05, 0.95), 
             xycoords='axes fraction',
             fontsize=12,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.tight_layout()
plt.show()

In [ ]:
# 特雷诺比率比较可视化
plt.figure(figsize=(10, 6))

# 创建条形图
assets = ['AAPL', 'SPY (市场)']
treynor_values = [annual_treynor_aapl, annual_treynor_spy]
colors = ['blue', 'green']

# 绘制条形图
bars = plt.bar(assets, treynor_values, color=colors, width=0.6)

# 添加条形图上的数值标签
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.002,
             f'{height:.4f}',
             ha='center', va='bottom', fontsize=12)

# 设置图表属性
plt.title('AAPL与市场(SPY)的年化特雷诺比率比较', fontsize=16)
plt.ylabel('年化特雷诺比率', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.3, axis='y')

# 添加注释解释
plt.figtext(0.15, 0.02, 
            f"注: 特雷诺比率 = (资产收益率 - 无风险收益率) / Beta\nAAPL Beta: {beta_aapl:.4f}, 市场(SPY) Beta: 1.00", 
            ha="left", fontsize=10,
            bbox={"facecolor":"white", "alpha":0.9, "pad":5})

plt.tight_layout(padding=2)
plt.show()

In [ ]:
# 累计收益率对比
# 计算累计收益率
data_aapl_aligned['Cumulative Return'] = (1 + data_aapl_aligned['Daily Return']).cumprod()
data_spy_aligned['Cumulative Return'] = (1 + data_spy_aligned['Daily Return']).cumprod()

plt.figure(figsize=(12, 6))
plt.plot(data_aapl_aligned.index, data_aapl_aligned['Cumulative Return'], 'b-', linewidth=2, label='AAPL累计收益')
plt.plot(data_spy_aligned.index, data_spy_aligned['Cumulative Return'], 'g-', linewidth=2, label='SPY累计收益')
plt.title('AAPL与SPY累计收益率对比', fontsize=16)
plt.xlabel('日期', fontsize=12)
plt.ylabel('累计收益率', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# 添加注释
plt.annotate(f"AAPL Beta: {beta_aapl:.4f}\nAAPL Treynor: {annual_treynor_aapl:.4f}\nSPY Treynor: {annual_treynor_spy:.4f}", 
             xy=(0.05, 0.95), 
             xycoords='axes fraction',
             fontsize=12,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.tight_layout()
plt.show()

## 1.8 结果分析与解释

通过上述计算和可视化，我们可以得出以下结论：

1. **Beta系数解读**：
   - AAPL的Beta值为{beta_aapl:.4f}
   - Beta > 1表示AAPL的波动性大于市场，对市场变动的反应更为敏感
   - Beta < 1表示AAPL的波动性小于市场，相对稳定
   - 在本例中，AAPL的Beta {'大于' if beta_aapl > 1 else '小于'} 1，说明它的系统性风险{'高于' if beta_aapl > 1 else '低于'}市场平均水平

2. **特雷诺比率比较**：
   - AAPL的年化特雷诺比率为{annual_treynor_aapl:.4f}
   - SPY的年化特雷诺比率为{annual_treynor_spy:.4f}
   - AAPL的特雷诺比率{'高于' if annual_treynor_aapl > annual_treynor_spy else '低于'}市场，表明在考虑系统性风险后，AAPL{'提供了更好的' if annual_treynor_aapl > annual_treynor_spy else '提供了较差的'}风险调整收益

3. **回归分析结果**：
   - 回归方程：AAPL收益率 = {model.params[0]:.6f} + {model.params[1]:.4f} × SPY收益率
   - R²值为{model.rsquared:.4f}，表明市场收益率能解释AAPL收益率变动的{model.rsquared:.2%}
   - 这意味着AAPL的收益有{(1-model.rsquared):.2%}来自非系统性因素（公司特有因素）

4. **投资策略建议**：
   - 特雷诺比率主要关注投资的系统性风险，适合用于评估已经分散化的投资组合
   - 如果投资者主要关注系统性风险，并且{'已经通过' if annual_treynor_aapl > annual_treynor_spy else '希望通过'}投资AAPL降低非系统性风险，则可以{'考虑' if annual_treynor_aapl > annual_treynor_spy else '谨慎考虑'}投资AAPL
   - 需要注意的是，特雷诺比率假设非系统性风险已经通过分散投资消除，因此单一股票的特雷诺比率应该结合其他指标一起考虑

5. **与夏普比率的对比**：
   - 与夏普比率不同，特雷诺比率只考虑系统性风险(Beta)，而忽略了总风险（标准差）
   - 特雷诺比率更适合评估已经分散投资的组合，而夏普比率适用于评估任何类型的投资
   - 对于单一资产，同时考虑特雷诺比率和夏普比率能提供更全面的风险收益评估